In [1]:
# dfs concurrency
import requests
from bs4 import BeautifulSoup
from collections import deque
import re
import threading

def dfs(url):
    visited = set([url])
    dq = deque([[url, "", 0]])
    max_depth = 3
    max_breadth = 5
    string = ""
    while dq:
        base, path, depth = dq.popleft()
        if depth < max_depth:
            try:
                soup = BeautifulSoup(requests.get(base + path).text, "html.parser")
                links = soup.find_all("a")[:max_breadth]
                for link in links:
                    href = link.get("href")
                    if href not in visited:
                        visited.add(href)
                        string += ("  " * depth + f"at depth {depth}: {href}\n")

                        if href.startswith("http") or href.startswith("https"):
                            dq.append([href, "", depth + 1])
                        else:
                            dq.append([base, href, depth + 1])
            except:
                pass

    filename =""
    if "https://" in url:
        filename = re.sub(r"https://","",url)
        filename = re.sub(r".com","",filename)
    elif "http://" in url:
        filename = re.sub(r"http://","",url)
        filename = re.sub(r".com","",filename)

    file  = open("crawled_links/"+filename+".txt","w")
    file.write(string)
    file.close()

In [2]:
frontier = ["http://toscrape.com", "https://soundcloud.com", "http://reddit.com", "https://fc2.com"]

def threadCreater(frontier):
    threads = []
    for i in frontier:
        t = threading.Thread(target = dfs, args=(i,))
        threads.append(t)
    return threads

threads = threadCreater(frontier)
for i in threads:
    i.start()
for i in threads:
    i.join()
print("files made!")

files made!


In [5]:
import math

def unary(x):
    return '0'*(x-1)+'1'

def encoder(x, b):
    q = math.floor(x/b)
    q_encode = unary(q+1)
    i = math.floor(math.log2(b))
    d = int(2**(i+1))-b
    r = x%b
    rem = ""
    if r < d:
        rem = bin(r)[2:]
        l = len(rem)
        if l < i:
            rem = '0'*(i - l)+rem
    else:
        rem = bin(r + d)[2:]
        l = len(rem)
        if l < i+1:
            rem = '0'*(i+1-l) + rem
    return q_encode+rem

print(encoder(74, 16))
print(encoder(50, 11))

000011010
000011011


In [13]:
def decode(n, M):
    q=len(n.split('1')[0])
    b=math.floor(math.log2(M))
    k = 2**(b+1)-M
    r = int(n[q+1:q+1+b],2)
    if r>=k:
        r = int(n[q+1:q+1+b+1],2)
        r=r-k
    x=q*M+r
    return x

print(decode("1111111110010001101", 10))

9


In [ ]:
# Q4
from nltk.tokenize import word_tokenize
from spacy.lang.en.stop_words import STOP_WORDS
from functools import reduce

stop_words = set(STOP_WORDS)
stop_words = stop_words.union({'.',',','\'','\"','?','{','}','[',']','<','>','(',')','!'})

def extract(url):
    html = requests.urlopen(url).read().decode('utf8')
    string = BeautifulSoup(html, 'html.parser').get_text()
    return list(word_tokenize(string))

def term_maker(ls, filenames):
    terms = []
    index = 0
    for i in ls:
        st = set(i).difference(stop_words)
        file = open('extracted_content/'+filenames[i]+'.txt','w')
        file.write(str(list(st)))
        file.close()
        index += 1
        terms.append(list(st))
    
    return terms

def counter(tokens, terms):
    ls = []
    for i in terms:
        ls.append(tokens.count(i))
    return ls